## match API
match API로 밴 픽 승 데이터를 가져옴

# 데이터 뽑을때만 쓰고 다른 때는 돌리지 마세요.

### 데이터 뽑을때는 버전과 csv 날짜를 꼭 확인해 주세요

In [1]:
import pandas as pd
import numpy as np
from time import sleep

VERSION = "10.23"

In [2]:
import os
for dirname,_,filenames in os.walk('./Data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

./Data\account_2020_10_08.csv
./Data\account_2020_11_03.csv
./Data\account_2020_11_23.csv
./Data\championName_ID.csv
./Data\matchData.json
./Data\matchID_2020_10_08.csv
./Data\matchID_2020_11_03.csv
./Data\matchID_2020_11_23.csv


In [3]:
matchData = pd.read_csv('./Data\matchID_2020_11_23.csv')

In [4]:
print(matchData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15447 entries, 0 to 15446
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   #        15447 non-null  int64
 1   matchID  15447 non-null  int64
dtypes: int64(2)
memory usage: 241.5 KB
None


In [5]:
print("Null value check")
print(matchData.isnull().sum())

Null value check
#          0
matchID    0
dtype: int64


## Request

## 데이터를 추출해서 DynamoDB에 저장


In [6]:
import requests
from roleidentification import pull_data, get_roles
import boto3
import traceback
import sys

URL = 'https://kr.api.riotgames.com/lol/match/v4/matches/'
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": #SECRET
}

trial=1

converter={}
data=pd.read_csv("./Data/championName_ID.csv")
for i,j in zip(data['id'],data['name']):
    converter[i]=j

print("start")
for ID in matchData['matchID']:
    try:
        URL_ID = URL+str(ID)
        res=requests.get(URL_ID,headers=headers)

        #error skip
        if(res.status_code!=200):
            #print(res.status_code,res.headers['X-APP-Rate-Limit-Count'])
            continue

        ## rate handling
        #print(res.headers['X-APP-Rate-Limit-Count'])
        if('100:120' in res.headers['X-APP-Rate-Limit-Count']):
            print(trial,"th Request Rate Waiting")
            trial=trial+1
            sleep(120)

        # if not rank match then continue
        if(res.json()['gameMode']!='CLASSIC'):
            #print(res.json()['gameMode'])
            continue

        banList=[]

        for team in res.json()['teams']:
            for ban in team['bans']:
                name=converter[ban['championId']]
                banList.append(name)

        #no ban means no rank
        if(banList==[]):
            #print("NO Ban")
            continue


        players=[]

        for participant in res.json()['participants']:
            player={}
            player['pickturn']=participant['participantId']
            name=converter[participant['championId']]
            player['championName']=name
            player["championID"]=participant['championId']
            #player['role']=participant['timeline']['role']
            player['lane']=participant['timeline']['lane']
            player['win']=participant['stats']['win']

            players.append(player)

        #lane manipulate
        champion_roles = pull_data()

        champs=[]

        #Red Team
        for j in range(5):
            champs.append(players[j]["championID"])
        roles=get_roles(champion_roles,champs)

        for j in range(5):
            for key in roles.keys():
                if(roles[key]==players[j]["championID"]):
                    players[j]['lane']=key

        #Blue Team
        champs=[]
        for j in range(5):
            champs.append(players[j+5]["championID"])
        roles=get_roles(champion_roles,champs)
        for j in range(5):
            for key in roles.keys():
                if(roles[key]==players[j+5]["championID"]):
                    players[j+5]['lane']=key

        tempdict={}
        tempdict['matchID']=(str)(ID)
        tempdict['ban']=banList
        tempdict['players']=players
        tempdict['version']= VERSION
        dynamodb = boto3.resource("dynamodb")
        table =dynamodb.Table("matchData")

        table.put_item(Item=tempdict)
        #match.append(tempdict)
    except Exception as e:
        traceback.print_exc()
        exctype, value = sys.exc_info()[:2]

#print(len(match))

print("End")

start
1 th Request Rate Waiting
2 th Request Rate Waiting
3 th Request Rate Waiting
4 th Request Rate Waiting
5 th Request Rate Waiting
6 th Request Rate Waiting
7 th Request Rate Waiting
8 th Request Rate Waiting
9 th Request Rate Waiting
10 th Request Rate Waiting
11 th Request Rate Waiting
12 th Request Rate Waiting
13 th Request Rate Waiting
14 th Request Rate Waiting
15 th Request Rate Waiting
16 th Request Rate Waiting
17 th Request Rate Waiting
18 th Request Rate Waiting
19 th Request Rate Waiting
20 th Request Rate Waiting
21 th Request Rate Waiting
22 th Request Rate Waiting
23 th Request Rate Waiting
24 th Request Rate Waiting
25 th Request Rate Waiting
26 th Request Rate Waiting
27 th Request Rate Waiting
28 th Request Rate Waiting
29 th Request Rate Waiting
30 th Request Rate Waiting
31 th Request Rate Waiting
32 th Request Rate Waiting
33 th Request Rate Waiting
34 th Request Rate Waiting
35 th Request Rate Waiting
36 th Request Rate Waiting
37 th Request Rate Waiting
38 t